# install packages

In [ ]:
pip install nbformat

In [49]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [50]:
import pandas as pd
import os
import pickle

In [ ]:
%run ./WalletClustering_neo4jConnect.ipynb # includes Neo4J connector
# methods & variables of notebook can be referenced

# query terror addresses

In [52]:
#The below code is returning a list of all the addresses that are marked as terror addresses.
#Query takes around 10 min
def returnTerrorAddresses():

    query_string = '''
    MATCH (a:Address {isTerror: True})
    Return a.address
    '''

    response = conn.query(query_string, db='neo4j')
    terrorAddresses = [r[0] for r in response]
    return terrorAddresses

In [53]:
# comment out entire block if old terrorAddressList saved as pickle should be used
def createTerrorAddressList():
    terrorAddressList = returnTerrorAddresses()

    # save terrorAddressList to file
    with open('terrorAddressList.pickle', 'wb') as export:
        pickle.dump(terrorAddressList, export)

In [54]:
# use existing blacklist if exists
if not os.path.exists('terrorAddressList.pickle'):
    createTerrorAddressList()

terrorAddressList = pickle.load(open('terrorAddressList.pickle', 'rb'))

# remove unwanted addresses

In [55]:
# # allows blacklist of addresses in json format. json file retrieved with https://github.com/nicotom/walletexplorer_scrapy
# 
# def createBlacklist():
#   dirname = os.path.dirname(os.path.realpath('__file__'))
#   folder = os.path.join(dirname, 'exchange_addresses\\json\\')
#   
#   def flatten(t):
#       return [item for sublist in t for item in sublist]
#   
#   blacklistAddresses = []
#   
#   # iterate over collection of exchange addresses in csv format to create blacklist
#   # attention, lists are not complete yet!
#   
#   for file in os.listdir(folder):
#       df = pd.read_json(os.path.join(folder, file))
#       blacklistAddresses.append(df['addresses'].tolist())
#       continue
#   
#   flatten(flatten(blacklistAddresses))

In [56]:
# flattens list of lists
def flatten(list):
    return [item for sublist in list for item in sublist]

In [57]:
# allows blacklist of addresses in csv format
blacklistAddresses = []
def createBlacklist():
    dirname = os.path.dirname(os.path.realpath('__file__'))
    folder = os.path.join(dirname, 'exchange_addresses\\csv\\')
   
    # iterate over collection of exchange addresses in csv format to create blacklist
    # attention, lists are not complete yet!
    for file in os.listdir(folder):
        df = pd.read_csv(os.path.join(folder, file), header=0)
        blacklistAddresses.append(df['hashAdd'].tolist())
        continue
    
    flatten(blacklistAddresses)

    # append known exchange addresses outside large dataset
    blacklistAddresses.append('13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc')
    
    #export blacklistAddresses to file
    with open('blacklistAddresses.pickle', 'wb') as export:
        pickle.dump(blacklistAddresses, export)

In [58]:
# use existing blacklist if exists
if not os.path.exists('blacklistAddresses.pickle'):
    createBlacklist()

blacklistAddresses = pickle.load(open('blacklistAddresses.pickle', 'rb'))



In [59]:
# to be used for removal of exchange addresses
interestingAddresses = []
def addressCleanUp(addressList, blacklist):
    for address in addressList:
        if address not in blacklist:
            interestingAddresses.append(address)
    return interestingAddresses

In [60]:
def createAddressesToClusterList():
    addressesToCluster = addressCleanUp(terrorAddressList, blacklistAddresses)

    # save addressesToCluster to file
    with open('addressesToCluster.pickle', 'wb') as export:
        pickle.dump(addressesToCluster, export)


In [61]:
# load addressesToCluster if exists
if not os.path.exists('addressesToCluster.pickle'):
    createAddressesToClusterList()

addressesToCluster = pickle.load(open('addressesToCluster.pickle', 'rb'))
